In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
#load files for google colab
from google.colab import files
uploaded = files.upload()

Saving data.ipynb to data.ipynb
Saving fin_data.ipynb to fin_data.ipynb
Saving models.ipynb to models.ipynb
Saving sp500.csv to sp500.csv


In [ ]:
!pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=4bfb46e55db2df03f7209e6fbd4bbeddf464a2a4380067c84658c1717af07ace
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
import import_ipynb
import tensorflow as tf
from fin_data import data_generator
from keras.models import Sequential
from keras.layers.merge import Add
from keras.optimizers import Adam,Nadam
from models import discriminator_model, generator_model_cnn,generator_model_mlp,generator_model_mlp_cnn,generator_model_mlp_cnn_plus
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
#import visualize
#import stylized_facts as sf
from datetime import datetime as dt
import os
#from shutil import copyfile
#import argparse 

# parser = argparse.ArgumentParser(description='FIN-GAN implementation')
# parser.add_argument('--batch_size',type=int,default=24)
# parser.add_argument('--generator_model',type=str,default='mlp-cnn')
# parser.add_argument('--epochs',type=int,default=10)
# parser.add_argument('--batches',type=int,default=1024)
# parser.add_argument('--folder_name',type=str,default='')
# parser.add_argument('--generator_lr',type=float,default='2e-4')
# parser.add_argument('--discriminator_lr',type=float,default='1e-5')
# parser.add_argument('--log_interval',type=int,default=50)
# parser.add_argument('--seed',type=int,default=1)

#args = parser.parse_args()
#seed(args.seed)
#set_random_seed(args.seed)


importing Jupyter notebook from fin_data.ipynb
importing Jupyter notebook from data.ipynb
importing Jupyter notebook from models.ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dg = data_generator()
batch_size = 24
dg.batch_size = batch_size
batches = 1024
epochs = 10            
timestamp = dt.now().strftime("%Y%m%d_%H%M%S")
timestamp += '_'
#timestamp += args.folder_name
# os.mkdir('./imgs/%s'%(timestamp))
# os.mkdir('./npy/%s'%(timestamp))
# os.mkdir('./weights/%s'%(timestamp))
# os.mkdir('./imgs/%s/acf'%(timestamp))
# os.mkdir('./imgs/%s/dist'%(timestamp))
# os.mkdir('./imgs/%s/time_series'%(timestamp))
# os.mkdir('./imgs/%s/leverage'%(timestamp))
# copyfile('./main.py','./imgs/%s/main.py'%(timestamp))
# copyfile('./models.py','./imgs/%s/models.py'%(timestamp))
# with open('./imgs/%s/hyper_parameters.txt'%(timestamp),'w') as w:
#     w.write(str(args))
def train():
    generator,generator_statistics = generator_model_mlp_cnn()

    #preparing generator
#     if args.generator_model == 'mlp-cnn':
#         generator,generator_statistics = generator_model_mlp_cnn()
#     elif args.generator_model == 'mlp':
#         generator,generator_statistics = generator_model_mlp()
#     elif args.generator_model == 'cnn':
#         generator = generator_model_cnn()
#     elif args.generator_model == 'plus':
#         generator = generator_model_mlp_cnn_plus()
#     else:
#         import sys
#         sys.exit()
    #preparing discriminator
                                                      
    statistics_opt = Adam(lr=0.0001)
    #generator_statistics.compile(loss='mean_squared_error',optimizer=statistics_opt)

    discriminator = discriminator_model()
    d_opt = Adam(lr=1e-5, beta_1=0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)
    discriminator.trainable = False
    for e in discriminator.layers:
        e.trainable = False
    gan = Sequential([generator,discriminator])
    g_opt = Adam(lr=2e-4, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=g_opt)
    
    g_loss_recorder = []
    d_loss_recorder = []

    #start training
    for epoch in range(epochs):
        for index in range(batches):
            noise = np.array([np.random.normal(0,1.0,size=100) for _ in range(batch_size)])
            real_series = dg.index_data_random_picker()
            real_series = np.nan_to_num(real_series)
            generated_series = generator.predict(noise, verbose=0)
#             if index % args.log_interval == 0:
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_abs_%i_%i'%(timestamp,epoch,index),for_abs=True)
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_raw_%i_%i'%(timestamp,epoch,index),for_abs=False)
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_abs_linear_%i_%i'%(timestamp,epoch,index),for_abs=True,scale='linear')
#                 sf.acf(generated_series,'./imgs/%s/acf/acf_raw_linear_%i_%i'%(timestamp,epoch,index),for_abs=False,scale='linear')
#                 sf.leverage_effect(generated_series,'./imgs/%s/leverage/leverage_%i_%i'%(timestamp,epoch,index))
#                 sf.distribution(generated_series, './imgs/%s/dist/distribution_%i_%i'%(timestamp,epoch,index),'linear')
#                 sf.distribution(generated_series, './imgs/%s/dist/distribution_%i_%i'%(timestamp,epoch,index),'log')
#                 visualize.time_series(generated_series[0],'./imgs/%s/time_series/generated_time_series_%i_%i'%(timestamp,epoch,index))
#                 np.save('./npy/%s/generated_time_series_%i_%i.npy'%(timestamp,epoch,index),generated_series)
            # update discriminator
            X = np.concatenate((real_series, generated_series))
            y = np.concatenate([np.random.uniform(0.9,1.1,batch_size),np.random.uniform(0.1,0.3,batch_size)])
            d_loss = discriminator.train_on_batch(X, y)
            d_loss_recorder.append(d_loss)
            # update generator
            y = np.array([1.]*batch_size,dtype=np.float32)
            g_loss = gan.train_on_batch(noise, y)
            g_loss_recorder.append(g_loss)
            print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))

    return generator, g_loss_recorder, d_loss_recorder

 #           generator.save_weights('./weights/%s/generator_%i_%i.h5'%(timestamp,epoch,index))

In [ ]:
generator, g_loss_arr, d_loss_arr = train() #started at high 0.6 for both generator and discriminator

Streaming output truncated to the last 5000 lines.
epoch: 5, batch: 120, g_loss: 0.608320, d_loss: 0.557792
epoch: 5, batch: 121, g_loss: 0.706654, d_loss: 0.576037
epoch: 5, batch: 122, g_loss: 0.644391, d_loss: 0.556546
epoch: 5, batch: 123, g_loss: 0.639303, d_loss: 0.555766
epoch: 5, batch: 124, g_loss: 0.675237, d_loss: 0.529347
epoch: 5, batch: 125, g_loss: 0.665345, d_loss: 0.608180
epoch: 5, batch: 126, g_loss: 0.561774, d_loss: 0.545397
epoch: 5, batch: 127, g_loss: 0.472271, d_loss: 0.516244
epoch: 5, batch: 128, g_loss: 0.535615, d_loss: 0.567560
epoch: 5, batch: 129, g_loss: 0.548619, d_loss: 0.576920
epoch: 5, batch: 130, g_loss: 0.604442, d_loss: 0.574546
epoch: 5, batch: 131, g_loss: 0.507575, d_loss: 0.551833
epoch: 5, batch: 132, g_loss: 0.495675, d_loss: 0.520526
epoch: 5, batch: 133, g_loss: 0.627079, d_loss: 0.503867
epoch: 5, batch: 134, g_loss: 0.557897, d_loss: 0.546989
epoch: 5, batch: 135, g_loss: 0.593480, d_loss: 0.547835
epoch: 5, batch: 136, g_loss: 0.67747

In [1]:
# plt.plot([i for i in range(len(g_loss_arr))], g_loss_arr)
# plt.plot([i for i in range(len(g_loss_arr))], d_loss_arr)

In [ ]:
noise = np.array([np.random.normal(0,1.0,size=100) for _ in range(batch_size)])
generated_series = generator.predict(noise, verbose=0) # compare different series to see if they are sufficiently different 
np.save('/content/gdrive/My Drive/GAN_paths.npy', generated_series)

In [ ]:
plt.plot((1+generated_series[0].flatten()).cumprod())
# plt.plot((1+generated_series[1].flatten()).cumprod())
# plt.plot((1+generated_series[2].flatten()).cumprod())
# plt.plot((1+generated_series[3].flatten()).cumprod())
# plt.plot((1+generated_series[4].flatten()).cumprod())
# plt.plot((1+generated_series[5].flatten()).cumprod())
# plt.plot((1+generated_series[6].flatten()).cumprod())
plt.plot((1+dg.index_data_random_picker()[0].flatten()).cumprod())